### Chapter 21. 클래스 메타 프로그래밍

클래스 메타프로그래밍은 실행 도중 클래스를 생성하거나 커스터마이징하는 기술이다. 매우 강력한 도구이지만 제대로 사용하기는 어렵고, 많은 경우 그렇게 추천되는 방식은 아니다. 프레임워크를 만들고 있는 게 아니라면 좋은 선택이 아니다.

+ 클래스 팩토리

적절한 기능의 클래스를 만들어낼 수 있는 클래스 팩토리가 있으면 유용할 때가 있다. 가령 개에 대한 데이터를 나타내고 싶다면 다음과 같은 코드를 생각할 수 있다.

In [ ]:
class Dog:
  def __init__(self, name, weight, owner):
    self.name=name
    self.weight=weight
    self.owner=owner

rex=Dog('rex', 30, 'Bob')
print(rex)

그러나 똑같은 필드명이 너무 많이 나오고 코드도 따분하다. 우리는 위의 Dog같은 간단한 클래스를 생성하는 클래스 팩토리를 만드는 것을 생각할 수 있다.

In [ ]:
def record_factory(cls_name, field_names):
  try:
    field_names=field_names.replace(","," ").split()
  except AttributeError: #replace나 split 사용불가
    field_names=tuple(field_names)

  def __init__(self, *args, **kwargs):
    attrs=dict(zip(self.__slots__, args))
    attrs.update(kwargs)
    for name, value in attrs.items():
      setattr(self,name,value)

  def __iter__(self):
    for name in self.__slots__:
      yield getattr(self,name)

  def __repr__(self):
    values=', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
    return '{}({})'.format(self.__class__.__name__, values)

  cls_attrs=dict(__slots__=field_names,
                 __init__=__init__,
                 __iter__=__iter__,
                 __repr__=__repr__)
  print(cls_attrs)
  
  return type(cls_name, (object,), cls_attrs)

Dog=record_factory('Dog', 'name weight owner')
#Dog 클래스를 만들어서 리턴한다
rex=Dog('Rex', 30, 'Bob')
print(rex, rex.weight)
print(rex.__slots__)
#rex의 속성들을 출력해줌

{'__slots__': ['name', 'weight', 'owner'], '__init__': <function record_factory.<locals>.__init__ at 0x7fefda4547b8>, '__iter__': <function record_factory.<locals>.__iter__ at 0x7fefd7665d08>, '__repr__': <function record_factory.<locals>.__repr__ at 0x7fefd7665ea0>}
Dog(name='Rex', weight=30, owner='Bob') 30
['name', 'weight', 'owner']


+ type 함수 알아보기

type(obj)는 객체의 클래스와 동일한 `obj.__class__` 를 가져오므로 type()를 함수로 생각할 수 있다. 그러나 type는 `name, bases, dict`를 인수로 받는 클래스이다. 인수 3개를 받아서 호출하면 새로운 클래스를 생성하는 일종의 클래스로서 작동한다. 아래 2개 코드는 완벽하게 같은 역할을 하는 것이다.

In [ ]:
MyClass=type('MyClass', (MySuperClass,), {'x':42, 'x2':lambda self:self.x*2})
#클래스의 이름, 클래스가 상속하는 슈퍼클래스, 클래스의 속성 딕셔너리를 받는다

In [ ]:
class MyClass(MySuperClass):
  x=42

  def x2(self):
    return self.x*2

이렇게 type 클래스를 이해하고 나면 위에서 클래스 팩토리를 만든 방식도 쉽게 이해할 수 있다. 이런 방식을 사용하면 type()에 인수 3개를 주어 호출하여 얼마든지 동적으로 클래스를 생성할 수 있다. 앞에서 보았던 namedtuple과도 비슷하지만 namedtuple은 exec함수를 사용하여 조금 다른 방식으로 구현되었다.

+ 클래스 데커레이터를 이용한 디스크립터 커스터마이즈

20장의 LineItem 예제는 잘 작동했지만 알아보기 쉬운 저장소명을 사용할 수 없었던 문제가 있었다. 

In [3]:
class Quantity:
  __counter=0

  def __init__(self):
    cls=self.__class__
    prefix=cls.__name__
    index=cls.__counter
    self.storage_name='_{}#{}'.format(prefix,index)
    #객체가 하나 더 생성될 때마다 storage_name의 해시번호를 1씩 늘려가면서 속성명으로 저장한다
    cls.__counter+=1

  def __get__(self, inst, owner):
    #관리대상 속성의 이름이 storage_name과 다르므로 __get__을 따로 구현해야 한다.
    #owner는 관리 대상 클래스-여기서는 LineItem-에 대한 참조이다
    if inst is None: #객체를 통해 호출 안하면 자기 자신을 반환
      return self
    else:
      return getattr(inst, self.storage_name)

  def __set__(self, inst, value):
    if value>0:
      setattr(inst, self.storage_name, value)
      #inst 안에 속성명과 값을 저장하자
    else:
      raise ValueError('value must be larger than 0')


class LineItem:
  weight=Quantity()
  price=Quantity()

  def __init__(self, description, weight, price):
    self.description=description
    self.weight=weight
    self.price=price

  
  def subtotal(self):
    return self.weight*self.price

coconut=LineItem('coconut', 20, 17.95)
print(coconut.weight, coconut.price)
print(LineItem.weight.storage_name)
#저장소명이 _Quantity#0 같은 알아보기 힘든 이름으로 되어있다

20 17.95
_Quantity#0


이런 문제는 디스크립터 객체가 생성되는 시점에는 관리 대상 속성-디스크립터 객체가 바인딩되는 속성-의 이름을 알 수 없기 때문에 발생한다. 이는 `__new__` 메서드를 사용해서도 해결할 수 있지만 컴퓨터 자원을 낭비하는 문제가 있다. 클래스를 생성하는 시점에 저장소명을 설정하는 식으로 해결하면 좋을 것이다. 이는 클래스 데커레이터를 써서 해결 가능하다.

클래스 데커레이터는 함수 데커레이터와 비슷하게 작동한다. 클래스를 받아서 동일하거나 수정된 클래스를 반환한다.
`cls=deco(cls)` 를 수행하는 것이다.

In [4]:
def entity(cls):
  print(cls.__dict__.items())
  for key, attr in cls.__dict__.items():
    if isinstance(attr,Quantity):
      type_name=type(attr).__name__
      attr.storage_name='_{}#{}'.format(type_name, key)
  return cls

@entity
class LineItem:
  weight=Quantity()
  price=Quantity()

  def __init__(self, description, weight, price):
    self.description=description
    self.weight=weight
    self.price=price

  
  def subtotal(self):
    return self.weight*self.price


raisin=LineItem('Gold raisin',10,6.95)
print(raisin)
print(LineItem.weight.storage_name)
#속성명이 들어간 storage_name이 잘 출력된다. 클래스가 생성될 때 데커레이터를 통해 미리 속성명을 다 받아와 놓았기 때문이다.

dict_items([('__module__', '__main__'), ('weight', <__main__.Quantity object at 0x7f0a38d3b630>), ('price', <__main__.Quantity object at 0x7f0a38d3b748>), ('__init__', <function LineItem.__init__ at 0x7f0a38d33e18>), ('subtotal', <function LineItem.subtotal at 0x7f0a38d33d90>), ('__dict__', <attribute '__dict__' of 'LineItem' objects>), ('__weakref__', <attribute '__weakref__' of 'LineItem' objects>), ('__doc__', None)])
_Quantity#weight


이처럼 클래스 데커레이터를 사용하면 클래스가 생성되는 순간의 클래스 커스터마이징을 쉽게 수행할 수 있다. 그러나 클래스 데커레이터는 자신에게 직접 적용된 클래스에서만 작동할 수 있다. 데커레이터가 적용된 클래스가 변경된 내용이 그 서브클래스에는 상속할 수도 아닐 수도 있는 것이다. 이를 해결하기 위해서는 메타클래스를 사용해야 한다. 메타클래스를 사용하면 클래스 계층구조 전체를 커스터마이즈할 수 있다.

+ 메타클래스 기본 지식

메타클래스는 일종의 클래스 팩토리지만 함수 대신 클래스로 만들어진다는 점이 다르다. 가령 파이썬 객체 모델을 생각하자. 클래스 자체도 객체이므로 각 클래스는 다른 어떤 클래스의 객체여야 한다. 그래서 기본적으로 파이썬 클래스는 type의 객체다. 즉 type는 대부분의 내장 클래스와 사용자 정의 클래스의 메타클래스인 것이다. 회귀를 방지하기 위해 type은 자기 자신의 객체로 정의되어 있다. 

In [ ]:
print('str'.__class__)
print(str.__class__)
print(object.__class__)
print(type.__class__)

<class 'str'>
<class 'type'>
<class 'type'>
<class 'type'>


str이나 type이 type클래스를 상속하는 것이 아니다. 모두 type의 객체일 뿐이다. str이나 type는 object의 서브클래스이다! object는 type의 객체이며 type는 object의 서브클래스이다. 이 관계는 마술과도 같아서 파이썬으로는 표현하기 힘들다. 일단 넘어간다.

이때 표준 라이브러리에는 Enum, ABCMeta 등의 다른 메타클래스도 있는데 이런 메타클래스들은 type을 상속하여 클래스 생성 능력을 이어받는다. 그래서 메타클래스들은 `__init__` 메서드를 구현해 자신의 객체를 커스터마이즈 할 수 있고, 클래스 데커레이터보다 훨씬 강력한 기능을 발휘한다.\

Tip) 메타클래스 구현시엔 관례적으로 self 대신 cls를 인수로 사용한다.

+ 메타클래스를 이용한 디스크립터 커스터마이징

그런데 LineItem 예제에서, 사용자가 데커레이터나 메타클래스에 신경쓰지 않고 단순히 라이브레리가 제공하는 클래스만 상속하게 할 수 있으면 좋을 것이다. 다음과 같이 구현할 수 있다.



In [5]:
class EntityMeta(type):
  """검증된 필드를 가진 비즈니스 개체 메타클래스"""
  def __init__(cls, name, bases, attr_dict):
    super().__init__(name, bases, attr_dict)
    for key, attr in attr_dict.items():
      if isinstance(attr, Quantity):
        type_name=type(attr).__name__ #클래스명을 붙여준다. 여기선 Quantity
        attr.storage_name='_{}#{}'.format(type_name, key) #Quantity#속성명 으로 붙음

class Entity(metaclass=EntityMeta):
  """검증된 필드를 가진 비즈니스 개체"""


class LineItem(Entity):
  weight=Quantity()
  price=Quantity()

  def __init__(self, description, weight, price):
    self.description=description
    self.weight=weight
    self.price=price

  
  def subtotal(self):
    return self.weight*self.price

+ 클래스 속성 순서 정의하기

가끔은 클래스의 속성이 정의되는 순서를 알아야 한다. 하지만 메타클래스의 `__new__` 와 `__init__` 메서드는 이름과 속성의 매핑으로 평가된 클래스 본체를 받는다. 매핑은 기본적으로 딕셔너리형이므로 속성의 순서가 사라져버리는 문제가 있다. 이를 해결하기 위해서는 `__prepare__` 메서드를 써야 한다. 이 메서드는 파이썬3에 소개되었으며, 메타클래스를 첫 번째 인수로 받고 생성할 클래스명과 슈퍼클래스가 담긴 튜플을 받으며 메타클래스의 `__new__, __init__` 의 마지막 인수로 전달할 매핑을 반환해야 한다. 즉, 속성을 저장할 객체를 미리 '준비' 해 놓는 메서드인 것이다. OrderedDict등을 사용하면 된다.

In [ ]:
import collections

class EntityMeta(type):
  
  @classmethod #클래스 메서드이므로 이 데커레이터가 필요하다
  def __prepare__(cls, name, bases):
    return collections.OrderedDict()
    #이제 클래스 속성은 이 __prepare__메서드가 반환한 OrderedDict 객체에 저장될 것이다. 따라서 속성에 순서가 생긴다.
    